# Rag ChatBot That Can Talk to Tables (.csv) files

In [1]:
data_path = "./assets-resources/superheroes.csv"

import pandas as pd

df = pd.read_csv(data_path)

In [2]:
df

,Superhero Name,Superpower,Power Level,Catchphrase
0,Captain Thunder,Bolt Manipulation,90,Feel the power of the storm!
1,Silver Falcon,Flight and Agility,85,"Soar high, fearlessly!"
2,Mystic Shadow,Invisibility and Illusions,78,Disappear into the darkness!
3,Blaze Runner,Pyrokinesis,88,Burn bright and fierce!
4,Electra-Wave,Electric Manipulation,82,Unleash the electric waves!
5,Crimson Cyclone,Super Speed,91,Blazing fast and unstoppable!
6,Aqua Fury,Hydrokinesis,80,Ride the waves of power!
7,Lunar Guardian,Lunar Manipulation,77,Embrace the moon's might!
8,Steel Titan,Super Strength and Durability,95,Indestructible force of nature!
9,Nightblade,Night Vision and Stealth,84,Strike from the shadows!


In [5]:
from langchain_community.document_loaders import CSVLoader

docs = CSVLoader(data_path).load_and_split()
docs

[Document(metadata={'source': './assets-resources/superheroes.csv', 'row': 0}, page_content='Superhero Name: Captain Thunder\nSuperpower: Bolt Manipulation\nPower Level: 90\nCatchphrase: Feel the power of the storm!'),
 Document(metadata={'source': './assets-resources/superheroes.csv', 'row': 1}, page_content='Superhero Name: Silver Falcon\nSuperpower: Flight and Agility\nPower Level: 85\nCatchphrase: Soar high, fearlessly!'),
 Document(metadata={'source': './assets-resources/superheroes.csv', 'row': 2}, page_content='Superhero Name: Mystic Shadow\nSuperpower: Invisibility and Illusions\nPower Level: 78\nCatchphrase: Disappear into the darkness!'),
 Document(metadata={'source': './assets-resources/superheroes.csv', 'row': 3}, page_content='Superhero Name: Blaze Runner\nSuperpower: Pyrokinesis\nPower Level: 88\nCatchphrase: Burn bright and fierce!'),
 Document(metadata={'source': './assets-resources/superheroes.csv', 'row': 4}, page_content='Superhero Name: Electra-Wave\nSuperpower: Ele

In [6]:
len(docs)

49

In [10]:
from langchain_chroma.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name="superhero-collection",
    persist_directory="./superhero-db"
)

In [15]:
from langchain.tools import tool
from langchain_core import vectorstores
from langchain.agents import create_agent

@tool
def retrieve_table_context(query: str):
    """
    Retrieves information from vectorstore with data about superhero db (.csv).
    """
    retrieved_docs = vector_store.similarity_search(query, k=5)
    return retrieved_docs

tools = [retrieve_table_context]

superhero_checker_agent = create_agent(
    model="openai:gpt-5-mini",
    system_prompt="You have access to a tool to retrieve context from a .csv table about superheroes information.",
    tools=tools
)

In [16]:
df.head()

,Superhero Name,Superpower,Power Level,Catchphrase
0,Captain Thunder,Bolt Manipulation,90,Feel the power of the storm!
1,Silver Falcon,Flight and Agility,85,"Soar high, fearlessly!"
2,Mystic Shadow,Invisibility and Illusions,78,Disappear into the darkness!
3,Blaze Runner,Pyrokinesis,88,Burn bright and fierce!
4,Electra-Wave,Electric Manipulation,82,Unleash the electric waves!


In [17]:
msg = "What is the superhero with the power of bolts?"
superhero_checker_agent.invoke(
    {"messages": [msg]}
)

{'messages': [HumanMessage(content='What is the superhero with the power of bolts?', additional_kwargs={}, response_metadata={}, id='cebcb133-4a35-4c8e-8499-afe6aa80d87b'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 163, 'total_tokens': 252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CnUB52zaSQbh2UatRJIwtTHnEy05R', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b2871-e3ad-7aa1-8773-7501cefd4f79-0', tool_calls=[{'name': 'retrieve_table_context', 'args': {'query': 'power bolts'}, 'id': 'call_eRMcFM6zohlRi81UJftBdOkS', 'type': 'tool_call'}], usage_metadata={'input_tokens': 163, 'ou